In [40]:
from Functions import *
from Optimizations import *
from utils import *
from Saving import *

%load_ext autoreload

%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [41]:
import time
import yaml

In [3]:
f = flat_sharp_gaussian
grad_f = grad_flat_sharp_gaussian

x_range = [-25, 25]

In [14]:
A = np.array([[1]])
b = np.array([0])
f = QuadraticFunctionInit(A, b)
grad_f = GradQuadraticFunctionInit(A)

x_range = [-50, 50]

In [ ]:
g_params = [[-10, 1, -2], [0, 10, -1], [10, 3, 1], [-20, 1, 3], [20, 1, 3]]
# g_params = [[-10, 0.7, -1], [0, 10, -1.5], [9, 3, 1], [17.5, 10, -0.5], [-20, 1, 3], [25, 1, 3]]
# g_params = [[-40, 0.7, -2],[-30, 0.7, -1.5],[-10, 0.7, -1], [33, 0.7, -1], 
#             [-20, 25, -2.5], [0, 10, -1.5], [9, 3, 1], [17.5, 10, -0.5], 
#             [-50, 1, 3], [50, 1, 3]]
# g_params = [[-40, 0.7, -2],[-30, 0.7, -1.5],[-5, 0.7, -1], [35, 0.7, -1.5], 
#             [-17.5, 30, -3], [5, 10, -1.5], [14, 3, 1], [25, 10, -0.5], 
#             [-50, 1, 3], [50, 1, 3]]

process = {}

process["particle_init"] = {"name": "uniform", 
                            "num_particles": 10000}

process["potential_function"] = {"name": "gaussian", 
                                 "params": g_params}

process["total_iter"] = 100
process["tau"] = 500
process["x_range"] = [-20, 20]

process["gamma"] = 0.5
process["temperature"] = 0.5
process["epsilon"] = 0

process["weight_function"] = {"name": "norm"} 
process["resample_function"] = {"name": "softmax", "params": {"beta": -100}} 

process["domain_enforcer"] = {"name": "hyper_cube_enforcer", "params": {"strength": 0.2}}

all_paths = diffusion_resampling(process)


In [ ]:
# want to get order: big_timestep, tau_timestep, particle, point

f, grad_f = get_potential(process)

X = np.linspace(process["x_range"][0], process["x_range"][1], 200)
inp = np.array([X])
Y = f(inp)

inp = all_paths.reshape([np.product(all_paths.shape[:3]), all_paths.shape[-1]]).T
out = f(inp)
all_paths_proc = np.concatenate([inp.T, out.reshape([len(out), 1])], axis=1).reshape(all_paths.shape[0], all_paths.shape[1], all_paths.shape[2], 2)

In [66]:
# full process densities 
K = multi_gaussian(np.array([[0.6]]))

folder_name = "full_{}_{}".format(process["potential_function"]["name"], process["tau"])
saved_path = create_animation_1d_pictures_particles(all_paths_proc, X, Y, folder_name=folder_name, graph_details={"p_size": 3, #"density_function": None})
                                                                                      "density_function": 
                                                                                      lambda x, p: V(np.array([x]), K, p)})
    
create_animation(saved_path, "{}.mp4".format(folder_name), framerate=10)

time.sleep(3) # otherwise it deletes the images before getting the video
remove_png(saved_path)
save_config(saved_path, process)

In [ ]:
# only feynman kac after selection densities 
fk_all_paths_proc = all_paths_proc[:, :, 0, :]
fk_all_paths_proc = fk_all_paths_proc.reshape([fk_all_paths_proc.shape[0], fk_all_paths_proc.shape[1], 1, fk_all_paths_proc.shape[2]])

K = multi_gaussian(np.array([[0.6]]))

folder_name = "selection_{}_{}_{}".format(process["potential_function"]["name"], process["resample_function"]["params"]["beta"], process["tau"])
saved_path = create_animation_1d_pictures_particles(fk_all_paths_proc, X, Y, folder_name=folder_name, graph_details={"p_size": 3, #"density_function": None})
                                                                                      "density_function": 
                                                                                      lambda x, p: V(np.array([x]), K, p)})
    
create_animation(saved_path, "{}.mp4".format(folder_name), framerate=10)

time.sleep(3) # otherwise it deletes the images before getting the video
remove_png(saved_path)
save_config(saved_path, process)

In [ ]:
# only feynman kac after mutation densities 
fk_all_paths_proc = all_paths_proc[:, :, -1, :]
fk_all_paths_proc = fk_all_paths_proc.reshape([fk_all_paths_proc.shape[0], fk_all_paths_proc.shape[1], 1, fk_all_paths_proc.shape[2]])

K = multi_gaussian(np.array([[0.6]]))

folder_name = "mutation_{}_{}_{}".format(process["potential_function"]["name"], process["resample_function"]["params"]["beta"], process["tau"])
saved_path = create_animation_1d_pictures_particles(fk_all_paths_proc, X, Y, folder_name=folder_name, graph_details={"p_size": 3, #"density_function": None})
                                                                                      "density_function": 
                                                                                      lambda x, p: V(np.array([x]), K, p)})
    
create_animation(saved_path, "{}.mp4".format(folder_name), framerate=10)

time.sleep(3) # otherwise it deletes the images before getting the video
remove_png(saved_path)
save_config(saved_path, process)

### Things to note:

The beta value of the softmax turns out to be very important. Adjusting that value determines how much to value each respective shallow/flat regions. Seems like changing it can give you a nice stationary distribution around certain falt minima. 

Also the tau value is important. Letting the run run for too long will cause the particles to approach more or less the stationary distribution if ran with soley diffusion. 

Maybe penalize where you started from. 



In [1]:
def load_config(load_dir):
    with open(load_dir + "/process.yml", "r") as f:
        c = yaml.load(f)
    return c

c = load_config("./tmp/1d__1579540892.8095877")

NameError: name 'yaml' is not defined